## Business Problem

#### A big entrepreneur with large chain of hotels  in Canada is looking to expand his business in Toronto with a capital of $1250000.
#### He is looking for a data scientist who can pull up relevant data & identify the ideal location based on Machine 
#### learning techniques for opening a restaurant in Central Toronto 


### Data for addressing the business need

#### Four square location data is used for location services
#### Data with postal codes in Canada is pulled from Wikipedia using requests module
#### The data is massaged & filtered down to "Central Toronto" before applying the machine learning technique.

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [4]:

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = BeautifulSoup(source,'lxml')

In [6]:
stat_table = soup.find_all('table',class_='wikitable')

In [7]:
len(stat_table)

1

In [8]:
stat_table = stat_table[0]

In [9]:

list = []
for row in stat_table.find_all('tr'):
    for cell in row.find_all('td'):
        list.append(cell.text.replace("\n",""))

In [10]:
num = numpy.array(list)

In [11]:

reshape=num.reshape(289,3)
Headers = ['Postcode','Borough','Neighbourhood']
Canada_df = pd.DataFrame(reshape,columns=Headers)
Canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
Canada_df = Canada_df.set_index('Borough')
Canada_df.head()

,Postcode,Neighbourhood
Borough,,
Not assigned,M1A,Not assigned
Not assigned,M2A,Not assigned
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,Harbourfront


In [13]:
Canada_df = Canada_df.drop('Not assigned', axis=0)
Canada_df.head()

,Postcode,Neighbourhood
Borough,,
North York,M3A,Parkwoods
North York,M4A,Victoria Village
Downtown Toronto,M5A,Harbourfront
Downtown Toronto,M5A,Regent Park
North York,M6A,Lawrence Heights


In [14]:
Canada_df=Canada_df.reset_index()
Canada_df=Canada_df.reindex(columns=Headers)
Canada_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [15]:
Canada_df['Neighbourhood'] = numpy.where(Canada_df['Neighbourhood']=='Not assigned',Canada_df['Borough'],Canada_df['Neighbourhood'])
Canada_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [16]:
Canada_df = Canada_df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ",".join(x))
Canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
Canada_df1 = pd.read_csv('/Users/Sathar/Documents/MD/DS/Trainings/Coursera/Assignments/Capstone Project/Geospatial_Coordinates.csv')
Canada_df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
Canada_df1.rename(columns = {'Postal Code': 'Postcode'}, inplace = True)
Canada_df1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
Canada_Final = pd.merge(Canada_df, Canada_df1, how = 'inner', on = 'Postcode' )
Canada_Final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [83]:
Toronto_df = Canada_Final[Canada_Final['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [84]:
Toronto_df.shape

(38, 5)

In [21]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

### Determining the coordinates for Toronto

In [22]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geographical coordinates of Toronto are 43.7170226, -79.4197830350134.


### Visualising the map of Toronto

In [85]:

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Function to get venues for a specified location

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
CLIENT_ID = 'HLSQKTDXPGKIE0HXRCGVPTA2ZOBMIVX1RF3PS1BGVJUFPB2I' # your Foursquare ID
CLIENT_SECRET = '1FBGVUNEURL5CQNZIE0T5OYITTCQ34H5CBKRQNFWVCURPJJP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HLSQKTDXPGKIE0HXRCGVPTA2ZOBMIVX1RF3PS1BGVJUFPB2I
CLIENT_SECRET:1FBGVUNEURL5CQNZIE0T5OYITTCQ34H5CBKRQNFWVCURPJJP


In [87]:
LIMIT=30
Toronto_venues= getNearbyVenues(names=Toronto_df['Neighbourhood'], 
                                  latitudes=Toronto_df['Latitude'],
                                  longitudes=Toronto_df['Longitude'])

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [88]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton,Exhibition Place,Parkdale Village",18,18,18,18,18,18
Business reply mail Processing Centre969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown,Grange Park,Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16


In [89]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighborhood'] 

list = Toronto_onehot.columns[:-1].tolist()
# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.shape

(38, 187)

### Top 5 venues around each neighbourhood

In [91]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.10
1           Steakhouse  0.10
2                Hotel  0.07
3  American Restaurant  0.07
4     Asian Restaurant  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.10
1                Café  0.07
2      Farmers Market  0.07
3  Seafood Restaurant  0.07
4                 Pub  0.03


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.11
1                    Café  0.11
2          Breakfast Spot  0.11
3                     Gym  0.06
4  Furniture / Home Store  0.06


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2    Recording Studio  0.05
3          Smoke Shop  0.05
4       Burrito Place  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  f

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in numpy.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in numpy.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Steakhouse,Hotel,American Restaurant,Asian Restaurant,Coffee Shop,Neighborhood,Smoke Shop,Plaza,Seafood Restaurant
1,Berczy Park,Cocktail Bar,Seafood Restaurant,Farmers Market,Café,Belgian Restaurant,Steakhouse,Restaurant,Pub,Bistro,Comfort Food Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Coffee Shop,Café,Grocery Store,Furniture / Home Store,Bar,Burrito Place,Stadium,Italian Restaurant,Caribbean Restaurant
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Skate Park,Restaurant,Smoke Shop,Brewery,Recording Studio,Spa,Farmers Market,Fast Food Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
5,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Italian Restaurant,Café,Indian Restaurant,General Entertainment,Market,Bank,Japanese Restaurant,Diner
6,Central Bay Street,Coffee Shop,Chinese Restaurant,Café,Italian Restaurant,Spa,Gastropub,Steakhouse,Park,Miscellaneous Shop,Bubble Tea Shop
7,"Chinatown,Grange Park,Kensington Market",Café,Caribbean Restaurant,Mexican Restaurant,Comfort Food Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Cocktail Bar,Cheese Shop,Organic Grocery
8,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Coffee Shop,Diner,Convenience Store,Nightclub,Restaurant
9,Church and Wellesley,Burger Joint,Adult Boutique,Bookstore,Breakfast Spot,Restaurant,Bubble Tea Shop,Ramen Restaurant,Pub,Pizza Place,Piano Bar


### Clustering Process

In [104]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)
Toronto_grouped_clustering.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632
4,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [95]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 1, 2, 2, 2, 2, 1], dtype=int32)

In [96]:

Toronto_merged = Toronto_df

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Other Great Outdoors,Pub,Neighborhood,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Spa,Pub,Pizza Place,Juice Bar,Indian Restaurant,Health Food Store,Yoga Studio
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Sandwich Place,Park,Pet Store,Liquor Store,Brewery,Burger Joint,Burrito Place,Pub,Italian Restaurant,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Café,American Restaurant,Italian Restaurant,Bakery,Comfort Food Restaurant,Fish Market,Sandwich Place,Bookstore,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Dim Sum Restaurant,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = numpy.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(numpy.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
numbs = []
for i in range(0, Toronto_merged.shape[1]):
      numbs.append(i)
numbs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

### Cluster 1

In [99]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + numbs]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Central Toronto,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,0,Coffee Shop,Sandwich Place,Café,Pizza Place,French Restaurant,Indian Restaurant,BBQ Joint,Liquor Store,History Museum,Vegetarian / Vegan Restaurant
27,Downtown Toronto,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina


### Cluster 2

In [100]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + numbs]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Café,American Restaurant,Italian Restaurant,Bakery,Comfort Food Restaurant,Fish Market,Sandwich Place,Bookstore,Seafood Restaurant
4,Central Toronto,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Dim Sum Restaurant,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
9,Central Toronto,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1,Coffee Shop,Pub,Bagel Shop,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Convenience Store,American Restaurant,Pizza Place
11,Downtown Toronto,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,1,Restaurant,Coffee Shop,Italian Restaurant,Café,Indian Restaurant,General Entertainment,Market,Bank,Japanese Restaurant,Diner
12,Downtown Toronto,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Burger Joint,Adult Boutique,Bookstore,Breakfast Spot,Restaurant,Bubble Tea Shop,Ramen Restaurant,Pub,Pizza Place,Piano Bar
13,Downtown Toronto,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Mexican Restaurant,Gym / Fitness Center,Theater,Dessert Shop
15,Downtown Toronto,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Italian Restaurant,Gastropub,Japanese Restaurant,Hotel,Restaurant,Creperie,Diner,Middle Eastern Restaurant,Spa
17,Downtown Toronto,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Chinese Restaurant,Café,Italian Restaurant,Spa,Gastropub,Steakhouse,Park,Miscellaneous Shop,Bubble Tea Shop
21,Downtown Toronto,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,1,Café,Restaurant,Coffee Shop,Deli / Bodega,Hotel,Gastropub,New American Restaurant,Salad Place,Seafood Restaurant,Pub
23,Central Toronto,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


### Cluster 3

In [101]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + numbs]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Other Great Outdoors,Pub,Neighborhood,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
1,East Toronto,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Spa,Pub,Pizza Place,Juice Bar,Indian Restaurant,Health Food Store,Yoga Studio
2,East Toronto,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Sandwich Place,Park,Pet Store,Liquor Store,Brewery,Burger Joint,Burrito Place,Pub,Italian Restaurant,Fast Food Restaurant
5,Central Toronto,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Grocery Store,Park,Clothing Store,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Burger Joint,Dance Studio,Dessert Shop
6,Central Toronto,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Gift Shop,Italian Restaurant
7,Central Toronto,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Seafood Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Diner,Pharmacy
8,Central Toronto,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,2,Park,Playground,Intersection,Restaurant,Yoga Studio,Dance Studio,Dumpling Restaurant,Dog Run,Discount Store,Diner
10,Downtown Toronto,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Yoga Studio,Cuban Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
14,Downtown Toronto,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Café,Coffee Shop,Clothing Store,Diner,Mexican Restaurant,Burrito Place,Plaza,Music Venue,Beer Bar,Pizza Place
16,Downtown Toronto,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Cocktail Bar,Seafood Restaurant,Farmers Market,Café,Belgian Restaurant,Steakhouse,Restaurant,Pub,Bistro,Comfort Food Restaurant


### Cluster 4

In [102]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + numbs]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Pool,Garden,Yoga Studio,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant


### Cluster 5

In [103]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + numbs]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,4,Cocktail Bar,Restaurant,Seafood Restaurant,Farmers Market,Café,Food Truck,Belgian Restaurant,Beer Bar,Jazz Club,Italian Restaurant


In [109]:
Central_Toronto = Toronto_merged[Toronto_merged['Borough']=='Central Toronto']

In [112]:
Central_Toronto.reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Dim Sum Restaurant,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Grocery Store,Park,Clothing Store,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Burger Joint,Dance Studio,Dessert Shop
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Gift Shop,Italian Restaurant
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Seafood Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Diner,Pharmacy
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,2,Park,Playground,Intersection,Restaurant,Yoga Studio,Dance Studio,Dumpling Restaurant,Dog Run,Discount Store,Diner
5,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1,Coffee Shop,Pub,Bagel Shop,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Convenience Store,American Restaurant,Pizza Place
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Pool,Garden,Yoga Studio,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
7,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
8,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,0,Coffee Shop,Sandwich Place,Café,Pizza Place,French Restaurant,Indian Restaurant,BBQ Joint,Liquor Store,History Museum,Vegetarian / Vegan Restaurant


In [ ]:
Venues = []
